## Generate representative forecasts

This notebook produces a figure of forecasts representative of high and low skill for local weather and airport weather. Results are stored in `output/misc_figures/`

Start by importing necessary libraries and turning off warnings

In [ ]:
import os, sys, importlib, json

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.dates as mdates

from scipy.stats import nbinom as nbinom
import pandas as pd
import numpy as np

import itertools
sys.path.append( os.path.abspath(os.path.join('..')) )
import utils.forecast as forecast
import utils.plotting_functions as pf
import utils.utils as utils

import warnings
warnings.filterwarnings('ignore')

# Opening config file
f = open('../fpaths_config.json')
paths = json.load(f)

raw_data_path = paths['raw_data']
smoothed_data_path = paths['smoothed_data']
misc_figures_path = paths['misc_figures_path']


## Still shots all in one

Next we again want to plot still shots, but have locations as rows, and columns being 25th percentile and 75th percentile for local forecasts and 75th percentile for airport forecast

In [ ]:
##Try it with a broken axis
importlib.reload(forecast)
importlib.reload(pf)

#Obtain these results from calculate_coverages.ipynb
#Q1 and Q3 (in order) quantile indices for smoothed, local weather forecasts
#['Arboleda', 'site', False] 57 95
#['Playa', 'site', False] 123 196
#['La_Margarita', 'site', False] 62 196
#['Villodas', 'site', False] 12 193

#First and third quantile indices for smoothed, airport weather forecasts
#['Arboleda', 'airport', False] 71 171
#['Playa', 'airport', False] 69 196
#['La_Margarita', 'airport', False] 113 24
#['Villodas', 'airport', False] 31 190

# Format is site = local Q1, local Q3, airport Q1, airport Q3
dict_idxs = dict(arboleda = (57, 95, 71, 171),
                 playa = (123, 196, 69, 196),
                 la_margarita = (62, 196, 113, 24),
                 villodas = (12, 193, 31, 190))

sites = ['Arboleda', 'Playa', 'La_Margarita', 'Villodas']

#Avg. mosquitoes
avg_mosquitos = [8.718, 12.786, 1.488, 1.624]
#Controls the positioning of the site labels
rtos = [0.22, 0.32, 0.18, 0.45]

scaler_win = 13
forecast_win = 52

fig = plt.figure(figsize=(14,8))

gs_frame = fig.add_gridspec(4,1, figure=fig)

trap_col, nn_col = 'tab:orange', 'tab:blue'
#set alpha value for scaler points
scaler_alpha=0.35

#First add Arboleda and Playa plots
for row_idx in [0]:
    gs = gridspec.GridSpecFromSubplotSpec(1,4, subplot_spec=gs_frame[row_idx])
    site = sites[row_idx]
    idxs = dict_idxs[site.lower()]
    rto = rtos[row_idx]
    avg_mos = avg_mosquitos[row_idx]
    for col_idx, idx in zip(range(0,4), idxs):
        if col_idx > 1:
            airport_flag = True
        else:
            airport_flag = False
        x_forecast, u_forecast_quant, l_forecast_quant, x_scale, nn_scale, nn_forecast, ref, trap_lo_err, trap_up_err, date_list = pf.still_shot(airport_flag, site, idx, scaler_win, forecast_win, avg_mos)
        ax = fig.add_subplot(gs[col_idx])
        ax = pf.still_add_to_ax(ax, x_forecast, u_forecast_quant, l_forecast_quant, x_scale, nn_scale, nn_forecast, ref, trap_lo_err, trap_up_err, date_list, forecast_win)
        ax = pf.formatting_single_plt(ax, row_idx, col_idx, site, rto, ref, nn_forecast, forecast_win)
        col_idx += 1


#Then add La Margarita and Villodas plots
for row_idx in [1,2,3]:
    gs = gridspec.GridSpecFromSubplotSpec(2,4, subplot_spec=gs_frame[row_idx], hspace=0.07, height_ratios=[1,3])
    site = sites[row_idx]
    idxs = dict_idxs[site.lower()]
    rto = rtos[row_idx]
    avg_mos = avg_mosquitos[row_idx]


    for col_idx, idx in zip(range(0,4), idxs):
        if col_idx > 1:
            airport_flag = True
        else:
            airport_flag = False
        x_forecast, u_forecast_sd, l_forecast_sd, x_scale, nn_scale, nn_forecast, ref, trap_lo_err, trap_up_err, date_list = pf.still_shot(airport_flag, site, idx, scaler_win, forecast_win, avg_mos)
        ax_hi = fig.add_subplot(gs[0,col_idx])
        ax_lo = fig.add_subplot(gs[1,col_idx])
                
        ax_hi = pf.still_add_to_ax(ax_hi, x_forecast, u_forecast_sd, l_forecast_sd, x_scale, nn_scale, nn_forecast, ref, trap_lo_err, trap_up_err, date_list, forecast_win)
        ax_lo = pf.still_add_to_ax(ax_lo, x_forecast, u_forecast_sd, l_forecast_sd, x_scale, nn_scale, nn_forecast, ref, trap_lo_err, trap_up_err, date_list, forecast_win)
        
        ax_hi = pf.formatting_hi(ax_hi, col_idx, site, rto, ref, nn_forecast, forecast_win)
        ax_lo = pf.formatting_lo(ax_lo, row_idx, col_idx, site, rto, ref, nn_forecast, forecast_win)
        
        d = 0.5
        kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
              linestyle="none", color='k', mec='k', mew=1, clip_on=False)
        ax_hi.plot([0, 1], [0, 0], transform=ax_hi.transAxes, **kwargs)
        ax_lo.plot([0, 1], [1, 1], transform=ax_lo.transAxes, **kwargs)
    
fig.tight_layout()
fig.subplots_adjust(wspace=0.12, hspace=0.3)

fig.savefig('..{}/still_comparison.png'.format(misc_figures_path), bbox_inches='tight', dpi=300)
